In [1]:
%%bash
pip install -U scikit-image

Requirement already up-to-date: scikit-image in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: PyWavelets>=0.4.0 in /usr/local/lib/python2.7/dist-packages (from scikit-image)
Requirement already up-to-date: scipy>=0.17.0 in /usr/local/lib/python2.7/dist-packages (from scikit-image)
Requirement already up-to-date: networkx>=1.8 in /usr/local/lib/python2.7/dist-packages (from scikit-image)
Requirement already up-to-date: six>=1.7.3 in /usr/local/lib/python2.7/dist-packages (from scikit-image)
Requirement already up-to-date: matplotlib>=1.3.1 in /usr/local/lib/python2.7/dist-packages (from scikit-image)
Requirement already up-to-date: pillow>=2.1.0 in /usr/local/lib/python2.7/dist-packages (from scikit-image)
Requirement already up-to-date: numpy>=1.9.1 in /usr/local/lib/python2.7/dist-packages (from PyWavelets>=0.4.0->scikit-image)
Requirement already up-to-date: decorator>=4.1.0 in /usr/local/lib/python2.7/dist-packages (from networkx>=1.8->scikit-image)
Requirem

In [69]:
import csv
import collections
import pandas
from sklearn import svm,metrics
from __future__ import division
import os
import numpy as np
import struct
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from skimage import io; io.use_plugin('matplotlib')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import csv
import os
import shutil
import imghdr
from sklearn.externals import joblib
import datetime
import time

In [3]:
csvfilepath='/opt/pgr/appgrimage.csv'
trainfilepath='/opt/pgr/exp2/train.txt'
testfilepath='/opt/pgr/exp2/test.txt'
featureVectorDir='/opt/pgr/exp2/featureVectors-4096'

In [4]:
#Storing the filename and complaint type as list
with open(csvfilepath) as csvfile:
    csvReader=csv.reader(csvfile,delimiter=',')
    complainType=[]
    imageName=[]
    for row in csvReader:
        complainType.append(row[1])
        imageName.append(row[3])

In [5]:
count=collections.Counter(complainType)
print (count)

Counter({'Removal of Debris': 11449, 'Desilting of Drain': 6102, 'Removal of garbage': 1615, 'Non Burning of Street Lights': 1511, 'Stagnation of water': 1173, 'Improper Sweeping': 752, 'Over flowing of garbage bins': 716, 'Disposal of removed silt on the Road': 663, 'Absenteesim of sweepers': 596, 'UGD Over Flow': 583, 'Broken Bin': 575, 'Absenteesim of door to door garbage collector': 515, 'Unsanitary conditions on the road': 437, 'Pot hole fill up/Repairs to the damage surface': 420, 'Water pipe leakage': 401, 'Mosquito menace': 370, 'Unauthorised / Illegal construction': 361, 'Burning of garbage': 337, 'Stray Pigs': 335, 'Provision of garbage bin': 327, 'Illegal draining of sewage to SWD/Open site': 286, 'Encroachment on the public property': 279, 'Bio Medical waste/Health hazard waste removal': 277, 'Dog menace': 271, 'Obstruction of water flow': 259, 'Open defecation- free (ODF)': 237, 'Issues Related to Drinking Water Supply': 230, 'Complaints regarding public toilets': 199, 'Re

In [5]:
#Writing the frequencies of each label in csv file
with open('frequency.csv','w') as csvfile:
    fieldnames=['label','count']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    for key, value in count.items():
        writer.writerow([key,value])

In [6]:
%%bash
pwd
cd ..
pwd
ls

/opt/pgr/notebooksPGR
/opt/pgr
appgrimage.csv
bagofwords.py
copy.sh
copy_label_specific.py
copy_metadataImages.py
data
exp1
exp2
featurevectors.py
images_major
notebooksPGR
pgrimagelist.csv
recources


In [6]:
#Extracting feature vectors from images
#run /opt/pgr/featurevectors.py /opt/pgr/exp2/Images /opt/pgr/recources/deploy.prototxt /opt/pgr/recources/bvlc_reference_caffenet.caffemodel /opt/pgr/recources/ilsvrc_2012_mean.npy fc7 /opt/pgr/exp2/featureVectors-4096 

In [7]:
#test
#caffe.io.load_image('/opt/pgr/exp2/Images/ea51856b-1fbb-45ff-b121-b9478dceb74b')

In [7]:
#Creating dictionary of complaint type
classlabels={}
i=0
with open('frequency.csv','r') as csvfile:
    csvReader=csv.reader(csvfile,delimiter=',')
    for row in csvReader:
        if row[0]=='label':
            continue
        classlabels[row[0]]=i
        i+=1
        if i==16:
            break

In [8]:
print classlabels

{'Removal of garbage': 2, 'Pot hole fill up/Repairs to the damage surface': 13, 'Absenteesim of door to door garbage collector': 11, 'Improper Sweeping': 5, 'Broken Bin': 10, 'Stagnation of water': 4, 'Desilting of Drain': 1, 'Absenteesim of sweepers': 8, 'Unsanitary conditions on the road': 12, 'Removal of Debris': 0, 'Over flowing of garbage bins': 6, 'Water pipe leakage': 14, 'Non Burning of Street Lights': 3, 'Disposal of removed silt on the Road': 7, 'Mosquito menace': 15, 'UGD Over Flow': 9}


In [9]:
#Generating txt file contaning files with specific labels
def createLabelSpecifictxt(src,txtfile,csvFilePath,dest):
    List = open(txtfile).readlines()
    #print List
    data=[]
    
    for i in range(0,len(List)):
        data.append(List[i].strip('\n'))

    with open(dest,'w') as f:
        with open(csvFilePath) as csvfile:
            readCSV=csv.reader(csvfile,delimiter=',')
            labels=[]
            imageName=[]
            for row in readCSV:
                name=row[3]
                label=row[1]
                fullpath=os.path.join(src,name)
                if (os.path.isfile(fullpath) and (label in classlabels) and (name in data)):
                     f.write(name)
                     f.write('\n')

In [10]:
#Storing the extracted feature vectors in an matrix
def loadFeatureVectors(dirPath,txtfilepath, d):
    allFiles=[]
    n = sum(1 for line in open(txtfilepath))
    with open(txtfilepath) as fp:
        data = np.zeros([n, d])
        i=0
        for fileName in fp:
            fileName=fileName.strip('\n')
            allFiles.append(fileName)
            with open("%s/%s" % (dirPath, fileName), "rb") as inputFile:
                data[i] = struct.unpack('f'*d, inputFile.read())
                if(np.all(np.isfinite(data[i]))==False):
                    print i
                i+=1
    return (data, allFiles)

# Generating labels of the files with filename in given text file
def makelabels(csvfilepath,txtfilepath):
    labels=[]
    with open(txtfilepath) as fp:
        df=pandas.read_csv(csvfilepath,error_bad_lines=False,header=None)
        df.columns = ["City", "Complaint Type", "Date", "FilestoreId","FilePath","Comments"]
        #print df
        imageNametoComplainType=dict(zip(df.FilestoreId, df['Complaint Type']))
        print len(imageNametoComplainType)
        for imageName in fp:
                 imageName=imageName.strip('\n')
                 labels.append(classlabels[imageNametoComplainType[imageName]])
        #print classlabels[data["Complaint Type"]] 
    return labels



In [11]:
# Creating training set labels and data
createLabelSpecifictxt(featureVectorDir,trainfilepath,csvfilepath,'/opt/pgr/exp2/train_specific.txt')
data,imageNames=loadFeatureVectors(featureVectorDir,'/opt/pgr/exp2/train_specific.txt',d=4096)
labels=makelabels(csvfilepath,'/opt/pgr/exp2/train_specific.txt')

# Creating testing set labels and data
createLabelSpecifictxt(featureVectorDir,testfilepath,csvfilepath,'/opt/pgr/exp2/test_specific.txt')
dataTest,imageNamesTest=loadFeatureVectors(featureVectorDir,'/opt/pgr/exp2/test_specific.txt',d=4096)
labelsTest=makelabels(csvfilepath,'/opt/pgr/exp2/test_specific.txt')

Skipping line 4248: expected 6 fields, saw 7
Skipping line 4249: expected 6 fields, saw 7
Skipping line 6909: expected 6 fields, saw 7
Skipping line 6910: expected 6 fields, saw 7
Skipping line 6911: expected 6 fields, saw 7
Skipping line 6912: expected 6 fields, saw 7
Skipping line 11651: expected 6 fields, saw 7
Skipping line 11700: expected 6 fields, saw 7
Skipping line 11701: expected 6 fields, saw 7
Skipping line 11702: expected 6 fields, saw 7
Skipping line 11840: expected 6 fields, saw 7
Skipping line 23326: expected 6 fields, saw 7
Skipping line 24704: expected 6 fields, saw 7
Skipping line 24754: expected 6 fields, saw 7
Skipping line 24962: expected 6 fields, saw 7
Skipping line 25259: expected 6 fields, saw 7
Skipping line 25380: expected 6 fields, saw 7
Skipping line 25517: expected 6 fields, saw 7
Skipping line 25892: expected 6 fields, saw 7
Skipping line 29076: expected 6 fields, saw 7
Skipping line 31093: expected 6 fields, saw 7
Skipping line 31094: expected 6 fields, 

34496


Skipping line 4248: expected 6 fields, saw 7
Skipping line 4249: expected 6 fields, saw 7
Skipping line 6909: expected 6 fields, saw 7
Skipping line 6910: expected 6 fields, saw 7
Skipping line 6911: expected 6 fields, saw 7
Skipping line 6912: expected 6 fields, saw 7
Skipping line 11651: expected 6 fields, saw 7
Skipping line 11700: expected 6 fields, saw 7
Skipping line 11701: expected 6 fields, saw 7
Skipping line 11702: expected 6 fields, saw 7
Skipping line 11840: expected 6 fields, saw 7
Skipping line 23326: expected 6 fields, saw 7
Skipping line 24704: expected 6 fields, saw 7
Skipping line 24754: expected 6 fields, saw 7
Skipping line 24962: expected 6 fields, saw 7
Skipping line 25259: expected 6 fields, saw 7
Skipping line 25380: expected 6 fields, saw 7
Skipping line 25517: expected 6 fields, saw 7
Skipping line 25892: expected 6 fields, saw 7
Skipping line 29076: expected 6 fields, saw 7
Skipping line 31093: expected 6 fields, saw 7
Skipping line 31094: expected 6 fields, 

34496


In [12]:
count=collections.Counter(labels)
print count
print len(data),len(labels)

Counter({0: 4490, 1: 3036, 2: 1057, 3: 934, 4: 554, 6: 525, 9: 474, 10: 375, 8: 323, 5: 318, 13: 288, 12: 250, 7: 243, 15: 238, 14: 236, 11: 220})
13561 13561


# Two Class Classifier

In [13]:
# Store feature vectors with specific labels in matrix
def loadFeatureVectorsTwoClass(dirPath,txtfilepath,labels, d):
    n = sum(1 for line in open(txtfilepath))
    count=collections.Counter(labels)
    l=count[0]+count[1]
    data = np.zeros([l, d])
    labelsInTwoClass=[]
    with open(txtfilepath) as fp:
        itr=0
        for i, fileName in enumerate(fp):
            if labels[i]==0 or labels[i]==1:
                fileName=fileName.strip('\n')
                with open("%s/%s" % (dirPath, fileName), "rb") as inputFile:
                        data[itr] = struct.unpack('f'*d, inputFile.read())
                        itr+=1
                        labelsInTwoClass.append(labels[i])
    return (data, labelsInTwoClass)

In [14]:
dataInTwoClass,labelsInTwoClass=loadFeatureVectorsTwoClass(featureVectorDir,'/opt/pgr/exp2/train_specific.txt',labels,d=4096)
dataInTwoClassTest,labelsInTwoClassTest=loadFeatureVectorsTwoClass(featureVectorDir,'/opt/pgr/exp2/test_specific.txt',labelsTest,d=4096)

In [15]:
#count=collections.Counter(labelsTest)
#print count[0]
#print count
print (len(dataInTwoClass),len(labelsInTwoClass))
print (len(dataInTwoClassTest),len(labelsInTwoClassTest))

(7526, 7526)
(1883, 1883)


In [51]:
#Training the classifier
''''
clfin2class = svm.SVC(probability=True)
clfin2class.fit(dataInTwoClass, labelsInTwoClass)
''''

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
# Saving the classifier
#joblib.dump(clfin2class, 'twoclasssvm.pkl') 

In [16]:
# Loading the classifier
clfin2class = joblib.load('twoclasssvm.pkl')

In [17]:
clfin2class.score(dataInTwoClassTest,labelsInTwoClassTest)

0.88688263409453

In [18]:
# Predicted labels of the test data
predicted=clfin2class.predict(dataInTwoClassTest)

In [19]:
CM=metrics.confusion_matrix(labelsInTwoClassTest,predicted)
print CM

[[1009  103]
 [ 110  661]]


# RandomForest Classifier

In [20]:
clfRFin2class  = RandomForestClassifier(n_estimators=10, max_depth=5,min_samples_split=5, random_state=0)
clfRFin2class = clfRFin2class.fit(dataInTwoClass, labelsInTwoClass)

In [21]:
clfRFin2class.score(dataInTwoClassTest,labelsInTwoClassTest)

0.8141263940520446

# Two vs all

In [22]:

# Generating labels of the files with filename in given text file
def maketwoVsAlllabels(csvfilepath,txtfilepath):
    labels=[]
    with open(txtfilepath) as fp:
        df=pandas.read_csv(csvfilepath,error_bad_lines=False,header=None)
        df.columns = ["City", "Complaint Type", "Date", "FilestoreId","FilePath","Comments"]
        #print df
        imageNametoComplainType=dict(zip(df.FilestoreId, df['Complaint Type']))
        print len(imageNametoComplainType)
        for imageName in fp:
                 imageName=imageName.strip('\n')
                 complaint=imageNametoComplainType[imageName]
                 if complaint != 'Removal of Debris' and complaint != 'Desilting of Drain':
                    label=2
                 else: label=classlabels[imageNametoComplainType[imageName]]
                 labels.append(label)
        #print classlabels[data["Complaint Type"]] 
    return labels


In [23]:
twoVsAllLabels=maketwoVsAlllabels('/opt/pgr/appgrimage.csv','/opt/pgr/exp2/train_specific.txt')
twoVsAllLabelsTest=maketwoVsAlllabels('/opt/pgr/appgrimage.csv','/opt/pgr/exp2/test_specific.txt')

Skipping line 4248: expected 6 fields, saw 7
Skipping line 4249: expected 6 fields, saw 7
Skipping line 6909: expected 6 fields, saw 7
Skipping line 6910: expected 6 fields, saw 7
Skipping line 6911: expected 6 fields, saw 7
Skipping line 6912: expected 6 fields, saw 7
Skipping line 11651: expected 6 fields, saw 7
Skipping line 11700: expected 6 fields, saw 7
Skipping line 11701: expected 6 fields, saw 7
Skipping line 11702: expected 6 fields, saw 7
Skipping line 11840: expected 6 fields, saw 7
Skipping line 23326: expected 6 fields, saw 7
Skipping line 24704: expected 6 fields, saw 7
Skipping line 24754: expected 6 fields, saw 7
Skipping line 24962: expected 6 fields, saw 7
Skipping line 25259: expected 6 fields, saw 7
Skipping line 25380: expected 6 fields, saw 7
Skipping line 25517: expected 6 fields, saw 7
Skipping line 25892: expected 6 fields, saw 7
Skipping line 29076: expected 6 fields, saw 7
Skipping line 31093: expected 6 fields, saw 7
Skipping line 31094: expected 6 fields, 

34496
34496


In [24]:
#print twoVsAllLabels
print len(data),len(twoVsAllLabels)

13561 13561


In [25]:
# Three label categories  'Removal of Debris','Desilting of Drain' and others trained using
# onevsone svm classifier
print("start: ",time.time(), time.clock())
ovo= OneVsOneClassifier( svm.SVC(probability=True)).fit(data, twoVsAllLabels)
print("End: ",time.time(), time.clock())
# Time taken 109 min

In [26]:
# Saving the classifier
joblib.dump(ovo, 'twovsallsvm.pkl') 

['twovsallsvm.pkl']

In [28]:
ovo.score(dataTest,twoVsAllLabelsTest)

0.8091759205143191

In [29]:
predicted=ovo.predict(dataTest)

In [53]:
CM=metrics.confusion_matrix(twoVsAllLabelsTest,predicted)
print CM

[[ 920   62  130]
 [  76  527  168]
 [ 102  115 1322]]


In [64]:
axis0= CM.sum(axis=0)
axis1=CM.sum(axis=1)
D=CM.diagonal()

M=np.zeros([3,2])
for i in range(0,3):
    for j in range(0,2):
        if j==0:
            M[i][j]=D[i]/axis0[i]
        else: M[i][j]=D[i]/axis1[i]
row_labels=['Removal of Debris','Desilting of Drain','Others']
column_labels=['Precision','Recall']
df = pandas.DataFrame(M, columns=column_labels, index=row_labels)
print df

                    Precision    Recall
Removal of Debris    0.837887  0.827338
Desilting of Drain   0.748580  0.683528
Others               0.816049  0.858999


In [65]:
print("start: ",time.time(), time.clock())
clf2vsAll = svm.SVC(probability=True)
clf2vsAll.fit(data, twoVsAllLabels)
print("End: ",time.time(), time.clock())

('start: ', 1521183787.39348, 8004.262279)
('End: ', 1521189194.569147, 13409.700587)


In [66]:
# Saving the classifier
joblib.dump(clf2vsAll, 'clf2vsAllsvm.pkl') 

['clf2vsAllsvm.pkl']

In [67]:
clf2vsAll.score(dataTest,twoVsAllLabelsTest)

0.8091759205143191

# AdaBoost Classifier

In [71]:
print("start: ",time.time(), time.clock())
adb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
             max_depth=1, random_state=0).fit(data, twoVsAllLabels)
print("End: ",time.time(), time.clock())

('start: ', 1521190517.539705, 13632.086385)
('End: ', 1521190948.812945, 14063.34103)


In [73]:
adb.score(dataTest,twoVsAllLabelsTest)

0.7285213325540619

# OneVsAll for three classes

In [ ]:
print("start: ",time.time(), time.clock())
ova= OneVsRestClassifier( svm.SVC(probability=True)).fit(data, twoVsAllLabels)
print("End: ",time.time(), time.clock())

('start: ', 1521192211.118566, 14063.588329)
